# Sprint 9. Aprenentatge Supervisat - Regressions

## Exercici 1. Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../estructures_dataframe/DelayedFlights.csv", sep=",")

In [3]:
df = df.set_index('Unnamed: 0')

#Dropping Na
df = df.dropna()

In [4]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
6,2008,1,3,4,1937.0,1830,2037.0,1940,WN,509,N763SW,240.0,250.0,230.0,57.0,67.0,IND,LAS,1591,3.0,7.0,0,N,0,10.0,0.0,0.0,0.0,47.0
11,2008,1,3,4,1644.0,1510,1845.0,1725,WN,1333,N334SW,121.0,135.0,107.0,80.0,94.0,IND,MCO,828,6.0,8.0,0,N,0,8.0,0.0,0.0,0.0,72.0
16,2008,1,3,4,1452.0,1425,1640.0,1625,WN,675,N286WN,228.0,240.0,213.0,15.0,27.0,IND,PHX,1489,7.0,8.0,0,N,0,3.0,0.0,0.0,0.0,12.0
18,2008,1,3,4,1323.0,1255,1526.0,1510,WN,4,N674AA,123.0,135.0,110.0,16.0,28.0,IND,TPA,838,4.0,9.0,0,N,0,0.0,0.0,0.0,0.0,16.0


In [5]:
df.describe().apply(lambda s: s.apply('{0:.3f}'.format))

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000,1247486.000
mean,2008.000,6.065,15.725,3.980,1558.832,1487.949,1616.749,1652.458,2276.526,135.378,131.764,107.420,63.291,59.677,741.587,7.297,20.660,0.000,0.000,19.179,3.703,15.022,0.090,25.296
std,0.000,3.509,8.793,1.993,454.330,421.178,583.942,461.737,1997.547,72.296,69.532,68.175,60.754,59.869,559.364,6.033,16.680,0.000,0.000,43.546,21.492,33.833,2.023,42.055
min,2008.000,1.000,1.000,1.000,1.000,0.000,1.000,0.000,1.000,14.000,-21.000,0.000,15.000,6.000,11.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,2008.000,3.000,8.000,2.000,1232.000,1150.000,1326.000,1340.000,637.000,83.000,80.000,58.000,26.000,24.000,334.000,4.000,11.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,2008.000,6.000,16.000,4.000,1618.000,1529.000,1737.000,1722.000,1589.000,118.000,115.000,90.000,43.000,41.000,595.000,6.000,16.000,0.000,0.000,2.000,0.000,2.000,0.000,8.000
75%,2008.000,9.000,23.000,6.000,1924.000,1830.000,2048.000,2022.000,3676.000,167.000,161.000,136.000,79.000,75.000,972.000,8.000,24.000,0.000,0.000,21.000,0.000,15.000,0.000,33.000
max,2008.000,12.000,31.000,7.000,2400.000,2359.000,2400.000,2359.000,9741.000,1114.000,660.000,1091.000,2461.000,2467.000,4962.000,240.000,422.000,0.000,0.000,2436.000,1352.000,1357.000,392.000,1316.000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1247486 entries, 4 to 7009718
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               1247486 non-null  int64  
 1   Month              1247486 non-null  int64  
 2   DayofMonth         1247486 non-null  int64  
 3   DayOfWeek          1247486 non-null  int64  
 4   DepTime            1247486 non-null  float64
 5   CRSDepTime         1247486 non-null  int64  
 6   ArrTime            1247486 non-null  float64
 7   CRSArrTime         1247486 non-null  int64  
 8   UniqueCarrier      1247486 non-null  object 
 9   FlightNum          1247486 non-null  int64  
 10  TailNum            1247486 non-null  object 
 11  ActualElapsedTime  1247486 non-null  float64
 12  CRSElapsedTime     1247486 non-null  float64
 13  AirTime            1247486 non-null  float64
 14  ArrDelay           1247486 non-null  float64
 15  DepDelay           1247486 non-n

### Train test split

Com que els models de regressió lineal només accepten valors numèrics, seleccionem només les columnes que no son objecte i més endavant podem convertir les columnes objecte que escaigui mitjançant columnes dummy, per exemple.


#### Split

In [7]:
from sklearn.model_selection import train_test_split

X = df[df.columns[~df.columns.isin(['CancellationCode','UniqueCarrier','TailNum','Origin','Dest','ArrDelay'])]]

y = df['ArrDelay']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2) 

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1247486 entries, 4 to 7009718
Data columns (total 23 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               1247486 non-null  int64  
 1   Month              1247486 non-null  int64  
 2   DayofMonth         1247486 non-null  int64  
 3   DayOfWeek          1247486 non-null  int64  
 4   DepTime            1247486 non-null  float64
 5   CRSDepTime         1247486 non-null  int64  
 6   ArrTime            1247486 non-null  float64
 7   CRSArrTime         1247486 non-null  int64  
 8   FlightNum          1247486 non-null  int64  
 9   ActualElapsedTime  1247486 non-null  float64
 10  CRSElapsedTime     1247486 non-null  float64
 11  AirTime            1247486 non-null  float64
 12  DepDelay           1247486 non-null  float64
 13  Distance           1247486 non-null  int64  
 14  TaxiIn             1247486 non-null  float64
 15  TaxiOut            1247486 non-n

### Model 1 Regressió lineal

In [9]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

y_hat_lr = lr.predict(X_test)

### Model 2 ElasticNet

In [10]:
from sklearn.linear_model import ElasticNetCV

elnet = ElasticNetCV()
elnet.fit(X_train, y_train)

y_hat_elnet = elnet.predict(X_test)

### Model 3 Decision Tree

In [11]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, y_train)

y_hat_clf = clf.predict(X_test)

## Exercici 2. Compara’ls en base al MSE i al R2.

R-squared represents the fraction of variance of response variable captured by the regression model rather than the MSE which captures the residual error. Per tant, el model que explica millor la variança en ArrDelay es la regressió lineal, amb un R2 màxim d'1, i el que menys error residual genera (1.79e-25).  

L'ElasticNet ocupa la segona posició tant en R2 com en error residual. 

L'arbre de decisió és el que més error captura i menys variança explica.

In [12]:
from sklearn.metrics import r2_score
r2_1 = {}
r2_1['r2_LinearRegression'] = r2_score(y_test, y_hat_lr)
r2_1['r2_ElasticNet'] = r2_score(y_test, y_hat_elnet)
r2_1['r2_DecisionTree'] = r2_score(y_test, y_hat_clf)

r2_1

{'r2_LinearRegression': 1.0,
 'r2_ElasticNet': 0.9996474010990963,
 'r2_DecisionTree': 0.9959699375206533}

In [13]:
from sklearn.metrics import mean_squared_error
mse_1 = {}
mse_1['mse_LinearRegression'] = mean_squared_error(y_test, y_hat_lr)
mse_1['mse_ElasticNet'] = mean_squared_error(y_test, y_hat_elnet)
mse_1['mse_DecisionTree'] = mean_squared_error(y_test, y_hat_clf)

mse_1

{'mse_LinearRegression': 1.7948154788866446e-25,
 'mse_ElasticNet': 1.2834120259717667,
 'mse_DecisionTree': 14.668879109251376}

## Exercici 3. Entrena’ls utilitzant els diferents paràmetres que admeten.

Si coneixem els diferents paràmetres que admeten els models podem utilitzar Grid Search per descobrir amb quina combinació de paràmetres obtenem una millor performance en cada model. 

Amb la regressió lineal totes les opcions ens donen el mateix valor d'r2: 1.000. Per tant, la configuració estàndard de la regressio lineal amb Fit_intercept: True, normalize: False i positive: False ja ens serveix en aquest cas.

L'arbre de decissió i l'ElasticNet permeten modificar molts paràmetres però només se n'han modificat alguns. 
La millor combinació, pel que fa a l'R2 (0.999937), de l'ElasticNet és fit_intercept: True, l1_ratio: 1 i normalize: True} 

Pel que fa a l'arbre de decissió, el millor R2 obtingut és d'0.995364 amb els paràmetres min_samples_leaf: 2, min_samples_split: 3 i splitter: best. 

#### Regressió lineal

In [14]:
from sklearn.model_selection import GridSearchCV

#Creating the grid for gridsearch
param_grid = {}
param_grid['fit_intercept'] = [True, False]
param_grid['normalize'] = [True, False]
param_grid['positive'] = [True, False]

grid_search_lr = GridSearchCV(LinearRegression(), param_grid, cv=3, scoring='r2', error_score=0)
grid_result_lr = grid_search_lr.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result_lr.best_score_, grid_result_lr.best_params_))
means = grid_result_lr.cv_results_['mean_test_score']
stds = grid_result_lr.cv_results_['std_test_score']
params = grid_result_lr.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#Create dict 
losses = {}    
#Update the dict    
losses['GridSearch lr' + str(grid_result_lr.best_params_)] = grid_result_lr.best_score_
losses

Best: 1.000000 using {'fit_intercept': True, 'normalize': True, 'positive': True}
1.000000 (0.000000) with: {'fit_intercept': True, 'normalize': True, 'positive': True}
1.000000 (0.000000) with: {'fit_intercept': True, 'normalize': True, 'positive': False}
1.000000 (0.000000) with: {'fit_intercept': True, 'normalize': False, 'positive': True}
1.000000 (0.000000) with: {'fit_intercept': True, 'normalize': False, 'positive': False}
1.000000 (0.000000) with: {'fit_intercept': False, 'normalize': True, 'positive': True}
1.000000 (0.000000) with: {'fit_intercept': False, 'normalize': True, 'positive': False}
1.000000 (0.000000) with: {'fit_intercept': False, 'normalize': False, 'positive': True}
1.000000 (0.000000) with: {'fit_intercept': False, 'normalize': False, 'positive': False}


{"GridSearch lr{'fit_intercept': True, 'normalize': True, 'positive': True}": 1.0}

#### ElasticNet

In [15]:
param_grid = {}
param_grid['l1_ratio'] = [0, 0.5, 1]
param_grid['fit_intercept'] = [True, False]
param_grid['normalize'] = [True, False]

grid_search_elnet = GridSearchCV(ElasticNetCV(), param_grid, cv=3, scoring='r2', error_score=0)
grid_result_elnet = grid_search_elnet.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result_elnet.best_score_, grid_result_elnet.best_params_))
means = grid_result_elnet.cv_results_['mean_test_score']
stds = grid_result_elnet.cv_results_['std_test_score']
params = grid_result_elnet.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

    
losses['GridSearch ElNet'+ str(grid_result_elnet.best_params_)] = grid_result_elnet.best_score_
losses

Best: 0.999937 using {'fit_intercept': True, 'l1_ratio': 1, 'normalize': True}
0.000000 (0.000000) with: {'fit_intercept': True, 'l1_ratio': 0, 'normalize': True}
0.000000 (0.000000) with: {'fit_intercept': True, 'l1_ratio': 0, 'normalize': False}
0.067673 (0.000152) with: {'fit_intercept': True, 'l1_ratio': 0.5, 'normalize': True}
0.999633 (0.000015) with: {'fit_intercept': True, 'l1_ratio': 0.5, 'normalize': False}
0.999937 (0.000002) with: {'fit_intercept': True, 'l1_ratio': 1, 'normalize': True}
0.999909 (0.000003) with: {'fit_intercept': True, 'l1_ratio': 1, 'normalize': False}
0.000000 (0.000000) with: {'fit_intercept': False, 'l1_ratio': 0, 'normalize': True}
0.000000 (0.000000) with: {'fit_intercept': False, 'l1_ratio': 0, 'normalize': False}
0.946174 (0.000629) with: {'fit_intercept': False, 'l1_ratio': 0.5, 'normalize': True}
0.946174 (0.000629) with: {'fit_intercept': False, 'l1_ratio': 0.5, 'normalize': False}
0.955780 (0.000648) with: {'fit_intercept': False, 'l1_ratio': 1

{"GridSearch lr{'fit_intercept': True, 'normalize': True, 'positive': True}": 1.0,
 "GridSearch ElNet{'fit_intercept': True, 'l1_ratio': 1, 'normalize': True}": 0.9999370586650528}

#### Decision Tree

In [16]:
param_grid = {}
param_grid['splitter'] = ['best','random']
param_grid['min_samples_split'] = [1, 2, 3]
param_grid['min_samples_leaf'] = [0.5, 1, 2]

grid_search_tree = GridSearchCV(tree.DecisionTreeRegressor(), param_grid, cv=3, scoring='r2', error_score=0)
grid_result_tree = grid_search_tree.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result_tree.best_score_, grid_result_tree.best_params_))
means = grid_result_tree.cv_results_['mean_test_score']
stds = grid_result_tree.cv_results_['std_test_score']
params = grid_result_tree.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

    
losses['GridSearch tree'+ str(grid_result_tree.best_params_)] = grid_result_tree.best_score_
losses

Best: 0.995364 using {'min_samples_leaf': 2, 'min_samples_split': 3, 'splitter': 'best'}
0.000000 (0.000000) with: {'min_samples_leaf': 0.5, 'min_samples_split': 1, 'splitter': 'best'}
0.000000 (0.000000) with: {'min_samples_leaf': 0.5, 'min_samples_split': 1, 'splitter': 'random'}
-0.000002 (0.000002) with: {'min_samples_leaf': 0.5, 'min_samples_split': 2, 'splitter': 'best'}
-0.000002 (0.000002) with: {'min_samples_leaf': 0.5, 'min_samples_split': 2, 'splitter': 'random'}
-0.000002 (0.000002) with: {'min_samples_leaf': 0.5, 'min_samples_split': 3, 'splitter': 'best'}
-0.000002 (0.000002) with: {'min_samples_leaf': 0.5, 'min_samples_split': 3, 'splitter': 'random'}
0.000000 (0.000000) with: {'min_samples_leaf': 1, 'min_samples_split': 1, 'splitter': 'best'}
0.000000 (0.000000) with: {'min_samples_leaf': 1, 'min_samples_split': 1, 'splitter': 'random'}
0.994701 (0.000121) with: {'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
0.992537 (0.000844) with: {'min_samples_l

{"GridSearch lr{'fit_intercept': True, 'normalize': True, 'positive': True}": 1.0,
 "GridSearch ElNet{'fit_intercept': True, 'l1_ratio': 1, 'normalize': True}": 0.9999370586650528,
 "GridSearch tree{'min_samples_leaf': 2, 'min_samples_split': 3, 'splitter': 'best'}": 0.9953639190413193}

## Exercici 4. Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

En l'exercici anterior, en el cas de l'ElasticNet, per exemple, podem veure com tot i utilitzant els paràmetres per defecte del model (fit_intercept: True, l1_ratio: 0.5 i normalize: False}, el R2 score és lleugerament diferent a l'ElasticNet del primer exercici. Això es deu a que en el primer cas hem fet la validació només amb el test subset i en l'exercici 3 hem utilitzat GridSearch, que inclou cross validation, és a dir, validació interna. Quan s'utilitza la cross validation el rendiment sol ser una mica inferior al rendiment del test subset però ens permet evitar el sobreentrenament del model.
 
R2 Default params ElasticNet Exercici 1: **0.999647**

R2 Default params ElasticNet Exercici 3: **0.999633**

Com que el GridSearch no ofereix la mètrica MSE, però sí la neg MSE, que segons [aquesta font](https://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error) sembla ser el MSE però en negatiu, 
calculem el Neg MSE amb cross validation pero comparar-ne el rendiment:

MSE Default params ElasticNet Exercici 1: **1.283412**

MSE Default params ElasticNet amb CV: **1.357881**


In [17]:
param_grid = {}
param_grid['l1_ratio'] = [0.5]
param_grid['fit_intercept'] = [True]
param_grid['normalize'] = [False]

grid_search_elnet = GridSearchCV(ElasticNetCV(), param_grid, cv=3, scoring='neg_mean_squared_error', error_score=0)
grid_result_elnet = grid_search_elnet.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result_elnet.best_score_, grid_result_elnet.best_params_))
means = grid_result_elnet.cv_results_['mean_test_score']
stds = grid_result_elnet.cv_results_['std_test_score']
params = grid_result_elnet.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

NegMSE = {}    
NegMSE['GridSearch ElNet_NegMSE'+ str(grid_result_elnet.best_params_)] = grid_result_elnet.best_score_
NegMSE

Best: -1.357881 using {'fit_intercept': True, 'l1_ratio': 0.5, 'normalize': False}
-1.357881 (0.052518) with: {'fit_intercept': True, 'l1_ratio': 0.5, 'normalize': False}


{"GridSearch ElNet_NegMSE{'fit_intercept': True, 'l1_ratio': 0.5, 'normalize': False}": -1.3578813990409693}

## Exercici 5. Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

Com que els models de regressió lineal només accepten valors numèrics, mitjançant columnes dummy podem transformar les columnes objecte en columnes numèriques. Per fer-ho analitzem el valor que aporta cadascuna de les columnes numèriques:

- Cancellation code només té un valor, pel que no aporta valor al model i no cal incloure'l. 

- Com que UniqueCarrier només te 20 valors differents podem crear columnes dummy per convertir la columna d'objecte a integer. 

- Tailnum, Origin i Dest tenen tants valors no numèrics diferents que no els anem a utilitzar per a la predicció.

In [18]:
df.CancellationCode.nunique()

1

In [19]:
df.UniqueCarrier.nunique()

20

In [20]:
df.TailNum.nunique()

5349

#### Dummies

In [21]:
df_dum = pd.get_dummies(df['UniqueCarrier'], drop_first=True)

In [22]:
df2 = pd.concat([df,df_dum], axis=1)
df2.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AA,AQ,AS,B6,CO,DL,EV,F9,FL,HA,MQ,NW,OH,OO,UA,US,WN,XE,YV
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,2008,1,3,4,1937.0,1830,2037.0,1940,WN,509,N763SW,240.0,250.0,230.0,57.0,67.0,IND,LAS,1591,3.0,7.0,0,N,0,10.0,0.0,0.0,0.0,47.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
11,2008,1,3,4,1644.0,1510,1845.0,1725,WN,1333,N334SW,121.0,135.0,107.0,80.0,94.0,IND,MCO,828,6.0,8.0,0,N,0,8.0,0.0,0.0,0.0,72.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
16,2008,1,3,4,1452.0,1425,1640.0,1625,WN,675,N286WN,228.0,240.0,213.0,15.0,27.0,IND,PHX,1489,7.0,8.0,0,N,0,3.0,0.0,0.0,0.0,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
18,2008,1,3,4,1323.0,1255,1526.0,1510,WN,4,N674AA,123.0,135.0,110.0,16.0,28.0,IND,TPA,838,4.0,9.0,0,N,0,0.0,0.0,0.0,0.0,16.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


#### Split

In [23]:
X_plus = df2[df2.columns[~df2.columns.isin(['CancellationCode','UniqueCarrier','TailNum','Origin','Dest','ArrDelay'])]]

y_plus = df2['ArrDelay']

X_train_plus, X_test_plus, y_train_plus, y_test_plus = train_test_split(X_plus, y_plus, random_state=0, test_size=0.2) 

### Model 1 Regressió lineal

In [24]:
lr_plus = LinearRegression()
lr_plus.fit(X_train_plus, y_train_plus)

y_hat_lr_plus = lr_plus.predict(X_test_plus)

### Model 2 ElasticNet

In [25]:
elnet_plus = ElasticNetCV()
elnet_plus.fit(X_train_plus, y_train_plus)

y_hat_elnet_plus = elnet_plus.predict(X_test_plus)

### Model 3 Decision Tree

In [26]:
clf_plus = tree.DecisionTreeRegressor()
clf_plus.fit(X_train_plus, y_train_plus)

y_hat_clf_plus = clf_plus.predict(X_test_plus)

### Mètriques

In [33]:
# R2
r2_2 = {}
r2_2['r2_LinearRegressionPlus'] = r2_score(y_test_plus, y_hat_lr_plus)
r2_2['r2_ElasticNetPlus'] = r2_score(y_test_plus, y_hat_elnet_plus)
r2_2['r2_DecisionTreePlus'] = r2_score(y_test_plus, y_hat_clf_plus)

In [34]:
# MSE
mse_2 = {}
mse_2['mse_LinearRegressionPlus'] = mean_squared_error(y_test_plus, y_hat_lr_plus)
mse_2['mse_ElasticNetPlus'] = mean_squared_error(y_test_plus, y_hat_elnet_plus)
mse_2['mse_DecisionTreePlus'] = mean_squared_error(y_test_plus, y_hat_clf_plus)

#### Amb aquestes dades i les dels exercics anteriors podem veure si realmente hem millorat la predició afegint la columna UniqueCarrier

Pel que fa a l'R2, afegir més variables al model no suposa cap tipus de millora en cap dels 3 models. A més, pel que fa a l'MSE, excepte en el cas de l'ElasticNet, que es queda igual, en la resta de models l'MSE ha augmentat lleugerament. Per tant, podem afirmar, que incloure més variables en un model no significa que el model pugui predir millor. 

In [46]:
display(r2_1)
display(r2_2)

{'r2_LinearRegression': 1.0,
 'r2_ElasticNet': 0.9996474010990963,
 'r2_DecisionTree': 0.9959699375206533}

{'r2_LinearRegressionPlus': 1.0,
 'r2_ElasticNetPlus': 0.9996474010990963,
 'r2_DecisionTreePlus': 0.996055181100377}

In [47]:
display(mse_1)
display(mse_2)

{'mse_LinearRegression': 1.7948154788866446e-25,
 'mse_ElasticNet': 1.2834120259717667,
 'mse_DecisionTree': 14.668879109251376}

{'mse_LinearRegressionPlus': 5.687343899902766e-25,
 'mse_ElasticNetPlus': 1.2834120259717667,
 'mse_DecisionTreePlus': 14.35860407698659}

## Exercici 6. No utilitzis la variable DepDelay a l’hora de fer prediccions

Com que hi ha una correlació molt elevanda (0.95) entre DepDelay i ArrDelay, els models es basen en DepDelay per fer la predicció d'ArrDelay. Si eliminem DepDelay del model, l'R2 i MSE dels models canviaran: 

- Pel que fa a l'R2: el model de regressió lineal genera el mateix resultat màxim d'1.0, l'ElasticNet té un resultat lleugèrament inferior (0.99965 vs 0.99898) i l'arbre de decisión, lleugerament superior (0.99597 vs 0.99690) quan no utilitzem la variable DepDelay.

- Pel que fa a l'MSE: aquest augmenta lleugerament en el cas de la regressió lineal (1.795e-25 vs 2.957e-25) i de l'ElasticNet (1.28 vs 3.73), però disminueix en l'arbre de decisió (14.67 vs 11.25). 


In [51]:
# Correlació entre DepDelay i ArrDelay
df.corr()['DepDelay']['ArrDelay']

0.9503232811843663

#### Split

In [55]:
X_3 = df2[df2.columns[~df2.columns.isin(['CancellationCode','UniqueCarrier','TailNum','Origin','Dest','ArrDelay','DepDelay'])]]

y_3 = df2['ArrDelay']

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, random_state=0, test_size=0.2) 

#### Model 1 Regressió lineal

In [57]:
lr_3 = LinearRegression()
lr_3.fit(X_train, y_train)

y_hat_lr_3 = lr_3.predict(X_test_3)

r2_3 = {}
r2_3['r2_LinearRegression_3'] = r2_score(y_test_3, y_hat_lr_3)

mse_3 = {}
mse_3['mse_LinearRegression_3'] = mean_squared_error(y_test_3, y_hat_lr_3)

#### Model 2 ElasticNet

In [58]:
elnet_3 = ElasticNetCV()
elnet_3.fit(X_train_3, y_train_3)

y_hat_elnet_3 = elnet_3.predict(X_test_3)

r2_3['r2_ElasticNet_3'] = r2_score(y_test_3, y_hat_elnet_3)
mse_3['mse_ElasticNet_3'] = mean_squared_error(y_test_3, y_hat_elnet_3)


#### Model 3 Decision Tree

In [59]:
clf_3 = tree.DecisionTreeRegressor()
clf_3.fit(X_train_3, y_train_3)

y_hat_clf_3 = clf_3.predict(X_test_3)

r2_3['r2_DecisionTree_3'] = r2_score(y_test_3, y_hat_clf_3)
mse_3['mse_DecisionTree_3'] = mean_squared_error(y_test_3, y_hat_clf_3)

#### Mètriques

In [62]:
# R2 primer vs segon model
display('R2 amb DepDelay: ', r2_1)
display('R2 sense DepDelay: ', r2_3)

# MSE primer vs segon model
display('MSE amb DepDelay: ',mse_1)
display('MSE sense DepDelay: ',mse_3)

'R2 amb DepDelay: '

{'r2_LinearRegression': 1.0,
 'r2_ElasticNet': 0.9996474010990963,
 'r2_DecisionTree': 0.9959699375206533}

'R2 sense DepDelay: '

{'r2_LinearRegression_3': 1.0,
 'r2_ElasticNet_3': 0.998976306335247,
 'r2_DecisionTree_3': 0.99690827428613}

'MSE amb DepDelay: '

{'mse_LinearRegression': 1.7948154788866446e-25,
 'mse_ElasticNet': 1.2834120259717667,
 'mse_DecisionTree': 14.668879109251376}

'MSE sense DepDelay: '

{'mse_LinearRegression_3': 2.956598523806986e-25,
 'mse_ElasticNet_3': 3.7261056596826387,
 'mse_DecisionTree_3': 11.25346094958677}